In [3]:
import numpy as np
import pandas as pd
from datetime import datetime

In [7]:
#final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB non-indexed skyline','30GB grid index skyline','40GB non-indexed skyline','40GB grid index skyline','50GB non-indexed skyline','50GB grid index skyline','60GB non-indexed skyline','60GB grid index skyline','70GB non-indexed skyline','70GB grid index skyline','80GB non-indexed skyline','80GB grid index skyline'])
#final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB grid index skyline','40GB grid index skyline','50GB grid index skyline','60GB grid index skyline','70GB grid index skyline'])
final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['50GB grid indexing operation'])
final_data.index.name="Size(GB)"
#path_graphs='C:/Tejas/PESU/CCBD_researchh/New Tests/global_index_skyline/graphs/'

#files=['c_50gb_skyline_normal_','c_50gb_skyline_grid_','c_50gb_skyline_rtree_','c_60gb_skyline_normal_','c_60gb_skyline_grid_','c_60gb_skyline_rtree_','c_70gb_skyline_normal_','c_70gb_skyline_grid_','c_70gb_skyline_rtree_']
#files=['c_30gb_skyline','c_30gb_skyline_grid','c_40gb_skyline','c_40gb_skyline_grid','c_50gb_skyline','c_50gb_skyline_grid','c_60gb_skyline','c_60gb_skyline_grid','c_70gb_skyline','c_70gb_skyline_grid','c_80gb_skyline','c_80gb_skyline_grid']
files=['c_30gb_skyline_grid','c_40gb_skyline_grid','c_50gb_skyline_grid','c_60gb_skyline_grid','c_70gb_skyline_grid']
path="C:/Tejas/PESU/CCBD_researchh/New Tests/skyline_ssd/FINAL DATA CPU UTIL"+'/'
counter=0
#HDD stats
#tests=['test7_8vcores_10vcores','test8_8vcores_10vcores','test9_8vcores_10vcores_reduce1','test10_8vcores_10vcores_reduce2','test12_8vcores_10vcores_reduce3','test13_8vcores_10vcores_reduce3_11520MB']
tests=['test16_8vcores_10vcores_reduce1_12920MB']
#tests=['test16_8vcores_10vcores_reduce1_12920MB','test14_8vcores_10vcores_reduce1_11820MB']
all_tests=['test1','test2']
for test in tests:
    counter=0
    for data in final_data.index:
        for col in final_data.columns:
            final_data.loc[str(data),col]=[]
    #print(final_data)
    for test_num in all_tests:
        print(test_num)
        counter=0
        for data in final_data.index:
            for col in final_data.columns:
                final_data.loc[str(data),col]=[]
        for data in final_data.index:
            #open job counters
            fp=open(path+test+"/"+test_num+'/master/'+files[counter],'r')
            string=fp.read()
            
            #Look for map time
            map_time=int(string[string.find('Total time spent by all map tasks (ms)'):].split("\n")[0].split("=")[1])
            
            #Look for reduce time
            reduce_time=int(string[string.find('Total time spent by all reduce tasks (ms)'):].split("\n")[0].split("=")[1])
            
            #look for cputime
            cpu_time=int(string[string.find('CPU time spent (ms)'):].split("\n")[0].split("=")[1])
            
            #Look for total time
            total_time=float(int(string[string.find('Total time: '):].split("\n")[0].split(":")[1].strip().split()[0])/1000)
            
            #Look for start time
            start_time=string.split("\n")[0].split(" ")[0]+" "+string.split("\n")[0].split(" ")[1]
            start_job_time=string[string.find("Running job:"):].split("\n")[1].split(" ")[0]+" "+string[string.find("Running job:"):].split("\n")[1].split(" ")[1]
            d1=datetime.strptime(start_time,"%y/%m/%d %H:%M:%S")
            d2=datetime.strptime(start_job_time,"%y/%m/%d %H:%M:%S")
            time_skip=d2-d1
            #time_skip=int(time_skip.split(":")[0]*3600) + int(time_skip.split(":")[1]*60) +int(time_skip.split(":")[2]) 
            time_skip=time_skip.total_seconds()
            
            #Look for filter phase end time
            filter_end_time=string[:string.find("Spatial filter function matched with")].split("\n")[-1].split(" ")[0]+ " "+string[:string.find("Spatial filter function matched with")].split("\n")[-1].split(" ")[1]
            d2=datetime.strptime(filter_end_time,"%y/%m/%d %H:%M:%S")
            time_skip_filter_end=d2-d1
            time_skip_filter_end=time_skip_filter_end.total_seconds()
            
            
            #Look for map start time
            map_start_time=string[string.find("running in uber mode : false"):].split("\n")[1].split(" ")[0]+ " "+string[string.find("running in uber mode : false"):].split("\n")[1].split(" ")[1]
            d2=datetime.strptime(map_start_time,"%y/%m/%d %H:%M:%S")
            time_skip_map_start=d2-d1
            time_skip_map_start=time_skip_map_start.total_seconds()
            
            #Look for map end time
            map_end_time=string[:string.find("INFO mapreduce.Job:  map 100%")].split("\n")[-1].split(" ")[0]+ " "+string[:string.find("INFO mapreduce.Job:  map 100%")].split("\n")[-1].split(" ")[1]
            d2=datetime.strptime(map_end_time,"%y/%m/%d %H:%M:%S")
            time_skip_map_end=d2-d1
            time_skip_map_end=time_skip_map_end.total_seconds()
            
            #Look for application number
            application_number=string[string.find("Running job:"):].split("\n")[0].split(" ")[2]
            application_number='application'+application_number.split("_",maxsplit=1)[1]
            
            
            final_data.loc[str(data),'map_time(ms)'].append(map_time)
            final_data.loc[str(data),'reduce_time(ms)'].append(reduce_time)
            final_data.loc[str(data),'total_time(s)'].append(total_time)
            final_data.loc[str(data),'cpu_time(ms)'].append(cpu_time)
            final_data.loc[str(data),'diff_time(s)'].append(time_skip)
            final_data.loc[str(data),'map_start_diff(s)'].append(time_skip_map_start)
            final_data.loc[str(data),'map_end_diff(s)'].append(time_skip_map_end)
            final_data.loc[str(data),'filter_end_diff(s)'].append(time_skip_filter_end)
            final_data.loc[str(data),'application_number']=application_number

            counter=counter+1
        for data in final_data.index:
            final_data.loc[str(data),'map_time(ms)']=np.average(final_data.loc[str(data),'map_time(ms)'])
            final_data.loc[str(data),'reduce_time(ms)']=np.average(final_data.loc[str(data),'reduce_time(ms)'])
            final_data.loc[str(data),'total_time(s)']=np.average(final_data.loc[str(data),'total_time(s)'])
            final_data.loc[str(data),'cpu_time(ms)']=np.average(final_data.loc[str(data),'cpu_time(ms)'])
            final_data.loc[str(data),'diff_time(s)']=np.average(final_data.loc[str(data),'diff_time(s)'])
            final_data.loc[str(data),'map_start_diff(s)']=np.average(final_data.loc[str(data),'map_start_diff(s)'])
            final_data.loc[str(data),'map_end_diff(s)']=np.average(final_data.loc[str(data),'map_end_diff(s)'])
            final_data.loc[str(data),'filter_end_diff(s)']=np.average(final_data.loc[str(data),'filter_end_diff(s)'])
            #final_data.loc[str(data),'application_number']=np.average(final_data.loc[str(data),'application_number'])
        
        final_data.to_excel(path+'ssd_counters_'+test+"_"+str(test_num)+'.xlsx')
        print(final_data)

test1
                        map_time(ms) reduce_time(ms) total_time(s)  \
Size(GB)                                                             
30GB grid index skyline       459882           23305          58.2   
40GB grid index skyline       483697           28984        58.853   
50GB grid index skyline       554075           31692        62.062   
60GB grid index skyline       601586           29041        69.469   
70GB grid index skyline       640804           39060        73.034   

                        cpu_time(ms) diff_time(s) map_start_diff(s)  \
Size(GB)                                                              
30GB grid index skyline       256330           15                15   
40GB grid index skyline       287180           11                12   
50GB grid index skyline       334030            9                 9   
60GB grid index skyline       350640           14                14   
70GB grid index skyline       394420           13                13   

     

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
#For parsing Grid Indexing operation

#final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB non-indexed skyline','30GB grid index skyline','40GB non-indexed skyline','40GB grid index skyline','50GB non-indexed skyline','50GB grid index skyline','60GB non-indexed skyline','60GB grid index skyline','70GB non-indexed skyline','70GB grid index skyline','80GB non-indexed skyline','80GB grid index skyline'])
#final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB grid index skyline','40GB grid index skyline','50GB grid index skyline','60GB grid index skyline','70GB grid index skyline'])
final_data=pd.DataFrame(columns=['total_time(s)','application_number','sampling_time(s)'],index=['AREA_LM_2015'])
final_data.index.name="Size(GB)"
#path_graphs='C:/Tejas/PESU/CCBD_researchh/New Tests/global_index_skyline/graphs/'

#files=['c_50gb_skyline_normal_','c_50gb_skyline_grid_','c_50gb_skyline_rtree_','c_60gb_skyline_normal_','c_60gb_skyline_grid_','c_60gb_skyline_rtree_','c_70gb_skyline_normal_','c_70gb_skyline_grid_','c_70gb_skyline_rtree_']
#files=['c_30gb_skyline','c_30gb_skyline_grid','c_40gb_skyline','c_40gb_skyline_grid','c_50gb_skyline','c_50gb_skyline_grid','c_60gb_skyline','c_60gb_skyline_grid','c_70gb_skyline','c_70gb_skyline_grid','c_80gb_skyline','c_80gb_skyline_grid']
#files=['c_30gb_skyline_grid','c_40gb_skyline_grid','c_50gb_skyline_grid','c_60gb_skyline_grid','c_70gb_skyline_grid']
#iles=['c_30gb_index_grid','c_40gb_index_grid','c_50gb_index_grid','c_60gb_index_grid','c_70gb_index_grid']
files=['c_AREA_LM_2015_index_grid']

#path="F:/Recovery lenovo laptop/Tejas/PESU/CCBD_researchh/New Tests/grid_indexing/global_index_grid_indexing/FINAL DATA CPU UTIL"+'/'
path="F:/Recovery lenovo laptop/Tejas/PESU/CCBD_researchh/New Tests/real_datasets/indexing/"
counter=0
#HDD stats
#tests=['test7_8vcores_10vcores','test8_8vcores_10vcores','test9_8vcores_10vcores_reduce1','test10_8vcores_10vcores_reduce2','test12_8vcores_10vcores_reduce3','test13_8vcores_10vcores_reduce3_11520MB']
tests=['hdd']
#tests=['test16_8vcores_10vcores_reduce1_12920MB','test14_8vcores_10vcores_reduce1_11820MB']
all_tests=['test1']
for test in tests:
    counter=0
    for data in final_data.index:
        for col in final_data.columns:
            final_data.loc[str(data),col]=[]
    #print(final_data)
    for test_num in all_tests:
        print(test_num)
        counter=0
        for data in final_data.index:
            for col in final_data.columns:
                final_data.loc[str(data),col]=[]
        for data in final_data.index:
            #open job counters
            fp=open(path+test+"/"+test_num+'/master/'+files[counter],'r')
            string=fp.read()
            
            #Look for map time
#             map_time=int(string[string.find('Total time spent by all map tasks (ms)'):].split("\n")[0].split("=")[1])
            
            #Look for reduce time
#             reduce_time=int(string[string.find('Total time spent by all reduce tasks (ms)'):].split("\n")[0].split("=")[1])
            
            #look for cputime
#             cpu_time=int(string[string.find('CPU time spent (ms)'):].split("\n")[0].split("=")[1])
            
            #Look for total time
            total_time=float(int(string[string.find('Total indexing time in millis'):].split("\n")[0].split(" ")[-1])/1000)
            
            
#             Look for start time
            start_time=string.split("\n")[0].split(" ")[0]+" "+string.split("\n")[0].split(" ")[1]
#             start_job_time=string[string.find("Running job:"):].split("\n")[1].split(" ")[0]+" "+string[string.find("Running job:"):].split("\n")[1].split(" ")[1]
            d1=datetime.strptime(start_time,"%y/%m/%d %H:%M:%S")
             
            #Look for sampling time
            sampling_time=float(int(string[string.find('Total time for sampling in millis:'):].split("\n")[0].split(":")[1].strip())/1000)
            
            #Look for application number
            application_number=string[string.find("Running job:"):].split("\n")[0].split(" ")[2]
            application_number='application'+application_number.split("_",maxsplit=1)[1]
            
            
#             final_data.loc[str(data),'map_time(ms)'].append(map_time)
#             final_data.loc[str(data),'reduce_time(ms)'].append(reduce_time)
            final_data.loc[str(data),'total_time(s)'].append(total_time)
#             final_data.loc[str(data),'cpu_time(ms)'].append(cpu_time)
#             final_data.loc[str(data),'diff_time(s)'].append(time_skip)
#             final_data.loc[str(data),'map_start_diff(s)'].append(time_skip_map_start)
#             final_data.loc[str(data),'map_end_diff(s)'].append(time_skip_map_end)
#             final_data.loc[str(data),'filter_end_diff(s)'].append(time_skip_filter_end)
            final_data.loc[str(data),'application_number']=application_number
            final_data.loc[str(data),'sampling_time(s)']=sampling_time

            counter=counter+1
        for data in final_data.index:
#             final_data.loc[str(data),'map_time(ms)']=np.average(final_data.loc[str(data),'map_time(ms)'])
#             final_data.loc[str(data),'reduce_time(ms)']=np.average(final_data.loc[str(data),'reduce_time(ms)'])
            final_data.loc[str(data),'total_time(s)']=np.average(final_data.loc[str(data),'total_time(s)'])
            final_data.loc[str(data),'sampling_time(s)']=np.average(final_data.loc[str(data),'sampling_time(s)'])

#             final_data.loc[str(data),'cpu_time(ms)']=np.average(final_data.loc[str(data),'cpu_time(ms)'])
#             final_data.loc[str(data),'diff_time(s)']=np.average(final_data.loc[str(data),'diff_time(s)'])
#             final_data.loc[str(data),'map_start_diff(s)']=np.average(final_data.loc[str(data),'map_start_diff(s)'])
#             final_data.loc[str(data),'map_end_diff(s)']=np.average(final_data.loc[str(data),'map_end_diff(s)'])
#             final_data.loc[str(data),'filter_end_diff(s)']=np.average(final_data.loc[str(data),'filter_end_diff(s)'])
            #final_data.loc[str(data),'application_number']=np.average(final_data.loc[str(data),'application_number'])
        
        final_data.to_excel(path+'ssd_counters_'+test+"_"+str(test_num)+'_grid_index.xlsx')
        print(final_data)

test1
             total_time(s)             application_number sampling_time(s)
Size(GB)                                                                  
AREA_LM_2015        39.956  application1558636471958_0001            37.89


In [1]:
#Distributed Join
import numpy as np
import pandas as pd
from datetime import datetime

#final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB non-indexed skyline','30GB grid index skyline','40GB non-indexed skyline','40GB grid index skyline','50GB non-indexed skyline','50GB grid index skyline','60GB non-indexed skyline','60GB grid index skyline','70GB non-indexed skyline','70GB grid index skyline','80GB non-indexed skyline','80GB grid index skyline'])
#final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB grid index skyline','40GB grid index skyline','50GB grid index skyline','60GB grid index skyline','70GB grid index skyline'])
final_data=pd.DataFrame(columns=['total_time(s)','application_number','filter_end_diff(s)'],index=['200MB X 400MB Grid DJ'])
final_data.index.name="Size(GB)"
#path_graphs='C:/Tejas/PESU/CCBD_researchh/New Tests/global_index_skyline/graphs/'

#files=['c_50gb_skyline_normal_','c_50gb_skyline_grid_','c_50gb_skyline_rtree_','c_60gb_skyline_normal_','c_60gb_skyline_grid_','c_60gb_skyline_rtree_','c_70gb_skyline_normal_','c_70gb_skyline_grid_','c_70gb_skyline_rtree_']
#files=['c_30gb_skyline','c_30gb_skyline_grid','c_40gb_skyline','c_40gb_skyline_grid','c_50gb_skyline','c_50gb_skyline_grid','c_60gb_skyline','c_60gb_skyline_grid','c_70gb_skyline','c_70gb_skyline_grid','c_80gb_skyline','c_80gb_skyline_grid']
#files=['c_30gb_skyline_grid','c_40gb_skyline_grid','c_50gb_skyline_grid','c_60gb_skyline_grid','c_70gb_skyline_grid']
files=['c_200mb_400mb_dj_grid']
path="/media/ubuntu/Seagate Backup Plus Drive/Recovery lenovo laptop/Tejas/PESU/CCBD_researchh/New Tests/dj/global_index_dj_ssd/FINAL DATA CPU UTIL"+'/'
counter=0
#HDD stats
#tests=['test7_8vcores_10vcores','test8_8vcores_10vcores','test9_8vcores_10vcores_reduce1','test10_8vcores_10vcores_reduce2','test12_8vcores_10vcores_reduce3','test13_8vcores_10vcores_reduce3_11520MB']
tests=['test16_8vcores_10vcores_reduce1_12920MB']
#tests=['test16_8vcores_10vcores_reduce1_12920MB','test14_8vcores_10vcores_reduce1_11820MB']
all_tests=['test3']
for test in tests:
    counter=0
    for data in final_data.index:
        for col in final_data.columns:
            final_data.loc[str(data),col]=[]
    #print(final_data)
    for test_num in all_tests:
        print(test_num)
        counter=0
        for data in final_data.index:
            for col in final_data.columns:
                final_data.loc[str(data),col]=[]
        for data in final_data.index:
            #open job counters
            fp=open(path+test+"/"+test_num+'/master/'+files[counter],'r')
            string=fp.read()
            
            #Look for map time
#             map_time=int(string[string.find('Total time spent by all map tasks (ms)'):].split("\n")[0].split("=")[1])
            
            #Look for reduce time
#             reduce_time=int(string[string.find('Total time spent by all reduce tasks (ms)'):].split("\n")[0].split("=")[1])
            
            #look for cputime
#             cpu_time=int(string[string.find('CPU time spent (ms)'):].split("\n")[0].split("=")[1])
            
            #Look for total time
            total_time=float(int(string[string.find('Join time'):].split("\n")[0].split(" ")[-2])/1000)
            
            
#             Look for start time
            start_time=string.split("\n")[0].split(" ")[0]+" "+string.split("\n")[0].split(" ")[1]
#             start_job_time=string[string.find("Running job:"):].split("\n")[1].split(" ")[0]+" "+string[string.find("Running job:"):].split("\n")[1].split(" ")[1]
            d1=datetime.strptime(start_time,"%y/%m/%d %H:%M:%S")
             
            #Look for filter end time
            filter_end_time=string[:string.find('mapred.BinarySpatialInputFormat:')].split("\n")[-1].split(" ")[0]+" "+string[:string.find('mapred.BinarySpatialInputFormat:')].split("\n")[-1].split(" ")[1]
            d2=datetime.strptime(filter_end_time,"%y/%m/%d %H:%M:%S")
            time_skip_filter_end=d2-d1
            time_skip_filter_end=time_skip_filter_end.total_seconds()
            
            #Look for application number
            application_number=string[string.find("Running job:"):].split("\n")[0].split(" ")[2]
            application_number='application'+application_number.split("_",maxsplit=1)[1]
            
            
#             final_data.loc[str(data),'map_time(ms)'].append(map_time)
#             final_data.loc[str(data),'reduce_time(ms)'].append(reduce_time)
            final_data.loc[str(data),'total_time(s)'].append(total_time)
#             final_data.loc[str(data),'cpu_time(ms)'].append(cpu_time)
#             final_data.loc[str(data),'diff_time(s)'].append(time_skip)
#             final_data.loc[str(data),'map_start_diff(s)'].append(time_skip_map_start)
#             final_data.loc[str(data),'map_end_diff(s)'].append(time_skip_map_end)
#             final_data.loc[str(data),'filter_end_diff(s)'].append(time_skip_filter_end)
            final_data.loc[str(data),'application_number']=application_number
            final_data.loc[str(data),'filter_end_diff(s)']=time_skip_filter_end

            counter=counter+1
        for data in final_data.index:
#             final_data.loc[str(data),'map_time(ms)']=np.average(final_data.loc[str(data),'map_time(ms)'])
#             final_data.loc[str(data),'reduce_time(ms)']=np.average(final_data.loc[str(data),'reduce_time(ms)'])
            final_data.loc[str(data),'total_time(s)']=np.average(final_data.loc[str(data),'total_time(s)'])
            final_data.loc[str(data),'filter_end_diff(s)']=np.average(final_data.loc[str(data),'filter_end_diff(s)'])

#             final_data.loc[str(data),'cpu_time(ms)']=np.average(final_data.loc[str(data),'cpu_time(ms)'])
#             final_data.loc[str(data),'diff_time(s)']=np.average(final_data.loc[str(data),'diff_time(s)'])
#             final_data.loc[str(data),'map_start_diff(s)']=np.average(final_data.loc[str(data),'map_start_diff(s)'])
#             final_data.loc[str(data),'map_end_diff(s)']=np.average(final_data.loc[str(data),'map_end_diff(s)'])
#             final_data.loc[str(data),'filter_end_diff(s)']=np.average(final_data.loc[str(data),'filter_end_diff(s)'])
            #final_data.loc[str(data),'application_number']=np.average(final_data.loc[str(data),'application_number'])
        
        final_data.to_excel(path+'ssd_counters_'+test+"_"+str(test_num)+'_dj_grid.xlsx')
        print(final_data)

test3
                      total_time(s)             application_number  \
Size(GB)                                                             
200MB X 400MB Grid DJ       4398.48  application1553270797903_0009   

                      filter_end_diff(s)  
Size(GB)                                  
200MB X 400MB Grid DJ                  2  


In [4]:
#Convexhull
import numpy as np
import pandas as pd
from datetime import datetime

#final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB non-indexed skyline','30GB grid index skyline','40GB non-indexed skyline','40GB grid index skyline','50GB non-indexed skyline','50GB grid index skyline','60GB non-indexed skyline','60GB grid index skyline','70GB non-indexed skyline','70GB grid index skyline','80GB non-indexed skyline','80GB grid index skyline'])
#final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB grid index skyline','40GB grid index skyline','50GB grid index skyline','60GB grid index skyline','70GB grid index skyline'])
final_data=pd.DataFrame(columns=['map_time(ms)','reduce_time(ms)','total_time(s)','cpu_time(ms)','diff_time(s)','map_start_diff(s)','map_end_diff(s)','filter_end_diff(s)','application_number'],index=['30GB grid indexed convexhull','40GB grid indexed convexhull','60GB grid indexed convexhull','70GB grid indexed convexhull'])
final_data.index.name="Size(GB)"
#path_graphs='C:/Tejas/PESU/CCBD_researchh/New Tests/global_index_skyline/graphs/'

#files=['c_50gb_skyline_normal_','c_50gb_skyline_grid_','c_50gb_skyline_rtree_','c_60gb_skyline_normal_','c_60gb_skyline_grid_','c_60gb_skyline_rtree_','c_70gb_skyline_normal_','c_70gb_skyline_grid_','c_70gb_skyline_rtree_']
#files=['c_30gb_skyline','c_30gb_skyline_grid','c_40gb_skyline','c_40gb_skyline_grid','c_50gb_skyline','c_50gb_skyline_grid','c_60gb_skyline','c_60gb_skyline_grid','c_70gb_skyline','c_70gb_skyline_grid','c_80gb_skyline','c_80gb_skyline_grid']
#files=['c_30gb_skyline_grid','c_40gb_skyline_grid','c_50gb_skyline_grid','c_60gb_skyline_grid','c_70gb_skyline_grid']
files=['c_30gb_convexhull_grid','c_40gb_convexhull_grid','c_60gb_convexhull_grid','c_70gb_convexhull_grid']
path="D:/Recovery lenovo laptop/Tejas/PESU/CCBD_researchh/New Tests/convexhull/global_index_convexhull/FINAL DATA CPU UTIL"+'/'
counter=0
#HDD stats
#tests=['test7_8vcores_10vcores','test8_8vcores_10vcores','test9_8vcores_10vcores_reduce1','test10_8vcores_10vcores_reduce2','test12_8vcores_10vcores_reduce3','test13_8vcores_10vcores_reduce3_11520MB']
tests=['test16_8vcores_10vcores_reduce1_12920MB']
#tests=['test16_8vcores_10vcores_reduce1_12920MB','test14_8vcores_10vcores_reduce1_11820MB']
all_tests=['test1']
for test in tests:
    counter=0
    for data in final_data.index:
        for col in final_data.columns:
            final_data.loc[str(data),col]=[]
    #print(final_data)
    for test_num in all_tests:
        print(test_num)
        counter=0
        for data in final_data.index:
            for col in final_data.columns:
                final_data.loc[str(data),col]=[]
        for data in final_data.index:
            #open job counters
            fp=open(path+test+"/"+test_num+'/master/'+files[counter],'r')
            string=fp.read()
            
            #Look for map time
            map_time=int(string[string.find('Total time spent by all map tasks (ms)'):].split("\n")[0].split("=")[1])
            
            #Look for reduce time
            reduce_time=int(string[string.find('Total time spent by all reduce tasks (ms)'):].split("\n")[0].split("=")[1])
            
            #look for cputime
            cpu_time=int(string[string.find('CPU time spent (ms)'):].split("\n")[0].split("=")[1])
            
            #Look for total time
            total_time=float(int(string[string.find('Total time: '):].split("\n")[0].split(":")[1].strip().split()[0])/1000)
            
            #Look for start time
            start_time=string.split("\n")[0].split(" ")[0]+" "+string.split("\n")[0].split(" ")[1]
            start_job_time=string[string.find("Running job:"):].split("\n")[1].split(" ")[0]+" "+string[string.find("Running job:"):].split("\n")[1].split(" ")[1]
            d1=datetime.strptime(start_time,"%y/%m/%d %H:%M:%S")
            d2=datetime.strptime(start_job_time,"%y/%m/%d %H:%M:%S")
            time_skip=d2-d1
            #time_skip=int(time_skip.split(":")[0]*3600) + int(time_skip.split(":")[1]*60) +int(time_skip.split(":")[2]) 
            time_skip=time_skip.total_seconds()
            
            #Look for filter phase end time
            filter_end_time=string[:string.find("Spatial filter function matched with")].split("\n")[-1].split(" ")[0]+ " "+string[:string.find("Spatial filter function matched with")].split("\n")[-1].split(" ")[1]
            d2=datetime.strptime(filter_end_time,"%y/%m/%d %H:%M:%S")
            time_skip_filter_end=d2-d1
            time_skip_filter_end=time_skip_filter_end.total_seconds()
            
            
            #Look for map start time
            map_start_time=string[string.find("running in uber mode : false"):].split("\n")[1].split(" ")[0]+ " "+string[string.find("running in uber mode : false"):].split("\n")[1].split(" ")[1]
            d2=datetime.strptime(map_start_time,"%y/%m/%d %H:%M:%S")
            time_skip_map_start=d2-d1
            time_skip_map_start=time_skip_map_start.total_seconds()
            
            #Look for map end time
            map_end_time=string[:string.find("INFO mapreduce.Job:  map 100%")].split("\n")[-1].split(" ")[0]+ " "+string[:string.find("INFO mapreduce.Job:  map 100%")].split("\n")[-1].split(" ")[1]
            d2=datetime.strptime(map_end_time,"%y/%m/%d %H:%M:%S")
            time_skip_map_end=d2-d1
            time_skip_map_end=time_skip_map_end.total_seconds()
            
            #Look for application number
            application_number=string[string.find("Running job:"):].split("\n")[0].split(" ")[2]
            application_number='application'+application_number.split("_",maxsplit=1)[1]
            
            
            final_data.loc[str(data),'map_time(ms)'].append(map_time)
            final_data.loc[str(data),'reduce_time(ms)'].append(reduce_time)
            final_data.loc[str(data),'total_time(s)'].append(total_time)
            final_data.loc[str(data),'cpu_time(ms)'].append(cpu_time)
            final_data.loc[str(data),'diff_time(s)'].append(time_skip)
            final_data.loc[str(data),'map_start_diff(s)'].append(time_skip_map_start)
            final_data.loc[str(data),'map_end_diff(s)'].append(time_skip_map_end)
            final_data.loc[str(data),'filter_end_diff(s)'].append(time_skip_filter_end)
            final_data.loc[str(data),'application_number']=application_number

            counter=counter+1
        for data in final_data.index:
            final_data.loc[str(data),'map_time(ms)']=np.average(final_data.loc[str(data),'map_time(ms)'])
            final_data.loc[str(data),'reduce_time(ms)']=np.average(final_data.loc[str(data),'reduce_time(ms)'])
            final_data.loc[str(data),'total_time(s)']=np.average(final_data.loc[str(data),'total_time(s)'])
            final_data.loc[str(data),'cpu_time(ms)']=np.average(final_data.loc[str(data),'cpu_time(ms)'])
            final_data.loc[str(data),'diff_time(s)']=np.average(final_data.loc[str(data),'diff_time(s)'])
            final_data.loc[str(data),'map_start_diff(s)']=np.average(final_data.loc[str(data),'map_start_diff(s)'])
            final_data.loc[str(data),'map_end_diff(s)']=np.average(final_data.loc[str(data),'map_end_diff(s)'])
            final_data.loc[str(data),'filter_end_diff(s)']=np.average(final_data.loc[str(data),'filter_end_diff(s)'])
            #final_data.loc[str(data),'application_number']=np.average(final_data.loc[str(data),'application_number'])
        
        final_data.to_excel(path+'ssd_counters_'+test+"_"+str(test_num)+'_convexhull_grid.xlsx')
        print(final_data)

test1
                             map_time(ms) reduce_time(ms) total_time(s)  \
Size(GB)                                                                  
30GB grid indexed convexhull  1.03802e+06           52565        87.388   
40GB grid indexed convexhull  1.15592e+06           59542        94.807   
60GB grid indexed convexhull  1.44332e+06           81015       120.906   
70GB grid indexed convexhull  1.58794e+06           91246       127.318   

                             cpu_time(ms) diff_time(s) map_start_diff(s)  \
Size(GB)                                                                   
30GB grid indexed convexhull       528290           10                10   
40GB grid indexed convexhull       597770           10                10   
60GB grid indexed convexhull       742940           13                13   
70GB grid indexed convexhull       833200           10                10   

                             map_end_diff(s) filter_end_diff(s)  \
Size(GB)           